## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
# I had to set the absolute path to get the CSV to read
vacation_df = pd.read_csv("C:/Users/zobai/Class/world_weather_analysis/Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Springbok,ZA,76.53,clear sky,-29.66,17.89,Undulata Country Lodge
1,2,Sinnamary,GF,75.88,moderate rain,5.38,-52.95,ALETHEIA Appartement Sinnamary
2,3,Arraial Do Cabo,BR,75.20,overcast clouds,-22.97,-42.03,Pousada Porto Praia
3,8,Vaini,TO,78.96,broken clouds,-21.20,-175.20,Keleti Beach Resort
4,9,Atuona,PF,78.48,clear sky,-9.80,-139.03,Villa Enata


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Using a filter to get four cities in Australia
# I didn't set an index name so I'm indexing using the actual indices
vacation_filter = vacation_df[vacation_df["Country"] == "AU"]

vacation_filter

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
19,118,Mount Isa,AU,89.37,broken clouds,-20.73,139.50,ibis Styles Mt Isa Verona
53,283,Ballina,AU,75.16,broken clouds,-28.87,153.57,Ramada Hotel & Suites by Wyndham Ballina Byron
62,315,Maryborough,AU,80.11,scattered clouds,-25.53,152.70,Blue Shades Motel
84,399,Hervey Bay,AU,80.04,broken clouds,-25.30,152.85,Shelly Bay Resort
100,468,Alice Springs,AU,80.15,broken clouds,-23.70,133.88,Desert Palms Alice Springs
102,470,Bowen,AU,75.42,light rain,-20.02,148.23,Castle Motor Lodge Motel
128,590,Port Augusta,AU,85.71,overcast clouds,-32.50,137.77,Majestic Oasis Apartments
138,623,Broken Hill,AU,78.51,clear sky,-31.95,141.43,Daydream Motel & Cottages
144,654,Gladstone,AU,84.13,broken clouds,-23.85,151.25,Highpoint International Hotel
147,658,Mackay,AU,80.58,light rain,-21.15,149.20,International Lodge Motel


In [6]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.
# Not sure if I did Steps 5 and 6 corectly but I referenced:
# https://medium.com/geekculture/advanced-filtering-and-data-manipulation-in-pandas-loc-iloc-apply-and-lambda-95990df0ad
# https://towardsdatascience.com/effective-data-filtering-in-pandas-using-loc-40eb815455b6
vacation_start = vacation_df.loc[[128]]
vacation_end = vacation_df.loc[[128]]
vacation_stop1 = vacation_df.loc[[100]]
vacation_stop2 = vacation_df.loc[[19]]
vacation_stop3 = vacation_df.loc[[138]]

In [7]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start.loc[128, ["Lat", "Lng"]].to_numpy()
end = vacation_end.loc[128, ["Lat", "Lng"]].to_numpy()
stop1 = vacation_stop1.loc[100, ["Lat", "Lng"]].to_numpy()
stop2 = vacation_stop2.loc[19, ["Lat", "Lng"]].to_numpy()
stop3 = vacation_stop3.loc[138, ["Lat", "Lng"]].to_numpy()

In [8]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
all_stops = gmaps.directions_layer(start, end,
                                   waypoints=[stop1, stop2, stop3],
                                   travel_mode="DRIVING")
fig.add_layer(all_stops)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# 8. To create a marker layer map between the four cities.
# Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,590,Port Augusta,AU,85.71,overcast clouds,-32.50,137.77,Majestic Oasis Apartments
1,468,Alice Springs,AU,80.15,broken clouds,-23.70,133.88,Desert Palms Alice Springs
2,118,Mount Isa,AU,89.37,broken clouds,-20.73,139.50,ibis Styles Mt Isa Verona
3,623,Broken Hill,AU,78.51,clear sky,-31.95,141.43,Daydream Motel & Cottages


In [12]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))